In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/project/spark-3.2.1-bin-hadoop3.2"
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark App") \
    .config("spark.jars", "/project/postgresql-42.3.2.jar") \
    .getOrCreate()

In [4]:
postgres_uri = "jdbc:postgresql://demsin0166.czfwea5noxbs.eu-west-2.rds.amazonaws.com:5432/initialdatabase"
dbtable = "public.google"
user = "username"
password = "password"

In [13]:
# SQL query to get the final DataFrame with all the relevant columns
final_df = spark.read \
    .format("jdbc") \
    .option("url", postgres_uri) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .option("query", 
        """SELECT restaurant_list.restaurant_name, google.address, google.phone_number, google.website, hygiene_rating.hygiene_rating,
           tripadvisor.about, tripadvisor.cuisines, tripadvisor.features, tripadvisor.special_diets ,restaurant_list.restaurant_url as tripadvisor_url, tripadvisor.price_range as tripadvisor_price_range, tripadvisor.food_rating as tripadvisor_food_rating, tripadvisor.rating as tripadvisor_overall_rating,tripadvisor.value_rating as tripadvisor_value_rating,
           google.price_level as google_price_level, google.rating as google_rating, google.total_ratings as google_total_ratings,
           yelp_data.price as yelp_price, yelp_data.rating as yelp_rating, yelp_data.review_count as yelp_review_count
           FROM restaurant_list 
           INNER JOIN google on restaurant_list.restaurant_id=google.restaurant_id
           INNER JOIN tripadvisor on restaurant_list.restaurant_id=tripadvisor.restaurant_id
           LEFT JOIN hygiene_rating on restaurant_list.restaurant_id=hygiene_rating.restaurant_id
           LEFT JOIN yelp_data on restaurant_list.restaurant_id=yelp_data.restaurant_id
           """ 
           )\
    .load()

final_df.show()

+--------------------+--------------------+---------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------+--------------------------+------------------------+------------------+-------------+--------------------+----------+-----------+-----------------+
|     restaurant_name|             address|   phone_number|             website|hygiene_rating|               about|            cuisines|            features|       special_diets|     tripadvisor_url|tripadvisor_price_range|tripadvisor_food_rating|tripadvisor_overall_rating|tripadvisor_value_rating|google_price_level|google_rating|google_total_ratings|yelp_price|yelp_rating|yelp_review_count|
+--------------------+--------------------+---------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------

In [8]:
# convert the queried tables to Pandas DataFrame and save as CSV
final_dfcsv = final_df.to_pandas_on_spark()
final_dfcsv.to_csv('restaurant_data.csv')


final_dfcsv

,restaurant_name,address,phone_number,website,hygiene_rating,about,cuisines,features,special_diets,tripadvisor_url,tripadvisor_price_range,tripadvisor_food_rating,tripadvisor_overall_rating,tripadvisor_value_rating,google_price_level,google_rating,google_total_ratings,yelp_price,yelp_rating,yelp_review_count
0,Bonoo Indian Tapas,"675 Finchley Rd, London NW2 2JP, UK",020 7794 8899,http://www.bonoo.co.uk/,5,,"Wine Bar, Indian, Contemporary, Street Food",,"Vegetarian Friendly, Vegan Options, Gluten Fre...",https://www.tripadvisor.com/Restaurant_Review-...,?20 - ?30,5.0,5.0,4.5,2,4.9,1342,None,5.0,1.0
1,Hibox,"48 Goodge St, London W1T 4LX, UK",020 7580 9312,https://www.hibahibox.com/,1,Vegan Palestinian and Lebanese food for eat-in...,"Lebanese, Mediterranean, Middle Eastern, Fast ...","Delivery, Takeout, Seating, Accepts Credit Cards","Vegetarian Friendly, Vegan Options, Gluten Fre...",https://www.tripadvisor.com/Restaurant_Review-...,?6 - ?10,5.0,5.0,5.0,no price level,4.9,207,None,NaN,NaN
2,Scarlett Green,"4 Noel St, London W1F 8GB, UK",020 3653 2010,http://www.daisygreenfood.com/,5,"Quintessential heart of Soho, Scarlett Green h...","Vegetarian Friendly, Vegan Options, Gluten Fre...","Reservations, Outdoor Seating, Seating, Highch...","Breakfast, Lunch, Dinner, Brunch, Drinks",https://www.tripadvisor.com/Restaurant_Review-...,,4.5,5.0,4.5,2,4.4,656,None,4.0,5.0
3,Nora Cafe,"9 Wentworth St, London E1 7TB, UK",020 7247 4992,http://www.noracafe.co.uk/,5,Try the best; You must check out our exception...,"Vegetarian Friendly, Vegan Options","Takeout, Seating, Free Wifi, Accepts Credit Ca...","Breakfast, Lunch, Dinner, Brunch, Late Night, ...",https://www.tripadvisor.com/Restaurant_Review-...,,5.0,5.0,5.0,no price level,4.8,142,£,5.0,3.0
4,Bayleaf Restaurant,"1282-1284 High Rd, London N20 9HH, UK",020 8446 8671,http://www.bayleaf.co.uk/restaurant/,5,We have evolved our cuisine over the years to ...,"Indian, Asian","Takeout, Reservations, Seating, Highchairs Ava...","Vegetarian Friendly, Vegan Options, Gluten Fre...",https://www.tripadvisor.com/Restaurant_Review-...,?36 - ?90,4.5,4.5,4.0,2,4.5,292,££,3.0,3.0
5,Amrutha Lounge,"326 Garratt Ln, London SW18 4EJ, UK",020 8001 4628,http://www.amrutha.co.uk/,4,Innovative vegan dishes and heart-warming clas...,"Indian, Asian, Healthy","Delivery, Takeout, Reservations, Seating, High...","Vegetarian Friendly, Vegan Options, Gluten Fre...",https://www.tripadvisor.com/Restaurant_Review-...,?6 - ?30,5.0,5.0,5.0,1,4.9,879,None,5.0,3.0
6,Andy's Greek Taverna,"23 Pratt St., London NW1 0BG, UK",020 7485 9718,https://www.andystaverna.co.uk/,None,,"Mediterranean, European, Greek, Healthy","Vegetarian Friendly, Vegan Options, Gluten Fre...",Dinner,https://www.tripadvisor.com/Restaurant_Review-...,,4.5,5.0,4.5,2,4.6,1696,£,5.0,27.0
7,Muhib Indian Cuisine,"73 Brick Ln, London E1 6QL, UK",020 7247 7122,http://www.muhibindiancuisine.com/,4,,"Indian, Bangladeshi",,"Vegetarian Friendly, Vegan Options, Halal, Glu...",https://www.tripadvisor.com/Restaurant_Review-...,?14 - ?30,4.5,4.5,4.0,2,4.5,544,££,4.0,77.0
8,Chamisse,"55 Grays Inn Rd, London WC1X 8PP, UK",020 7430 0799,http://www.chamisse.com/,3,"A Friendly casual , yet chic Lebanese / Medite...","Lebanese, Mediterranean, Middle Eastern","Delivery, Takeout, Reservations, Seating, High...","Vegetarian Friendly, Vegan Options, Halal, Glu...",https://www.tripadvisor.com/Restaurant_Review-...,?8 - ?30,4.5,4.5,4.5,2,4.6,679,££,5.0,15.0
9,Spasso,"61 Walm Ln, London NW2 4QR, UK",020 8450 7722,http://spassolondon.co.uk/,4,Spasso is a unique Italian restaurant and bar ...,"Italian, Mediterranean, Neapolitan, Campania, ...","Takeout, Reservations, Outdoor Seating, Seatin...","Vegetarian Friendly, Vegan Options",https://www.tripadvisor.com/Restaurant_Review-...,?8 - ?48,5.0,5.0,5.0,no price level,4.9,208,None,NaN,NaN


In [ ]:
final_dfcsv.to_csv('final_df.csv')

In [18]:
# SQL query table with restaurant names and their respective reviews
reviews_df = spark.read \
    .format("jdbc") \
    .option("url", postgres_uri) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .option("query", 
        """SELECT restaurant_list.restaurant_name, google_reviews.author_name, google_reviews.author_url, google_reviews.rating, google_reviews.text
           FROM google 
           INNER JOIN google_reviews on google.google_id=google_reviews.google_id
           INNER JOIN restaurant_list on restaurant_list.restaurant_id=google.restaurant_id
           """ 
           )\
    .load()

reviews_df.show()

+--------------------+----------------+--------------------+------+--------------------+
|     restaurant_name|     author_name|          author_url|rating|                text|
+--------------------+----------------+--------------------+------+--------------------+
|Eastern Eye Balti...|   Stephen Mudge|https://www.googl...|     5|Took advantage of...|
|Eastern Eye Balti...|         Zachary|https://www.googl...|     5|Their menu  desig...|
|Eastern Eye Balti...|    Allison Anne|https://www.googl...|     5|The Best! Good at...|
|  Bonoo Indian Tapas|           Mr. A|https://www.googl...|     5|The staff at Bono...|
|  Bonoo Indian Tapas|             A M|https://www.googl...|     5|Happened to find ...|
|  Bonoo Indian Tapas|  Art from Heart|https://www.googl...|     5|This is the best ...|
|Eastern Eye Balti...|   Stephen Mudge|https://www.googl...|     5|Took advantage of...|
|Eastern Eye Balti...|         Zachary|https://www.googl...|     5|Their menu  desig...|
|Eastern Eye Balti...

In [20]:
# convert the queried tables to Pandas DataFrame and save as CSV
reviews_dfcsv = final_df.to_pandas_on_spark()
reviews_dfcsv.to_csv('reviews_data.csv')


In [21]:
reviews_dfcsv

,restaurant_name,author_name,author_url,rating,text
0,Eastern Eye Balti House,Stephen Mudge,https://www.google.com/maps/contrib/1078933577...,5,"Took advantage of the early evening ""happy hou..."
1,Eastern Eye Balti House,Zachary,https://www.google.com/maps/contrib/1037898459...,5,Their menu design was amazing. They have eve...
2,Eastern Eye Balti House,Allison Anne,https://www.google.com/maps/contrib/1081146835...,5,The Best! Good atmosphere. Hot and Spicy food ...
3,Bonoo Indian Tapas,Mr. A,https://www.google.com/maps/contrib/1043795044...,5,The staff at Bonoo were amazing from the jump....
4,Bonoo Indian Tapas,A M,https://www.google.com/maps/contrib/1070061382...,5,Happened to find this place by chance when loo...
5,Bonoo Indian Tapas,Art from Heart,https://www.google.com/maps/contrib/1102197603...,5,This is the best Indian restaurant in north Lo...
6,Eastern Eye Balti House,Stephen Mudge,https://www.google.com/maps/contrib/1078933577...,5,"Took advantage of the early evening ""happy hou..."
7,Eastern Eye Balti House,Zachary,https://www.google.com/maps/contrib/1037898459...,5,Their menu design was amazing. They have eve...
8,Eastern Eye Balti House,Allison Anne,https://www.google.com/maps/contrib/1081146835...,5,The Best! Good atmosphere. Hot and Spicy food ...
9,Scarlett Green,Ali Douglas,https://www.google.com/maps/contrib/1098803929...,5,The food was exquisite. Really high quality an...
